In [14]:
import csv
import time

start = time.time()

with open('pvsubset.csv') as f:
    ticketTable = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

print "Total " + str(len(ticketTable)) + " Tickets Data Import : " + str(time.time() - start) + "seconds elapsed."

Total 112068 Tickets Data Import : 1.81085300446seconds elapsed.


In [3]:
!head pvsubset.csv

Summons Number,Plate ID,Registration State,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Vehicle Expiration Date,House Number,Street Name,Vehicle Color,Vehicle Year
1363745270,GGY6450,99,7/9/2015,46,SDN,HONDA,20170602,142,MACDOUNGH ST,WHITE,2010
1363745293,KXD355,SC,7/9/2015,21,SUBN,CHEVR,20160288,331,LEXINGTON AVE,RED,0
1363745438,JCK7576,PA,7/9/2015,21,SDN,ME/BE,0,1087,FULTON ST,WHITE,0
1363745475,GYK7658,NY,7/9/2015,21,SUBN,NISSA,0,207,MADISON ST,BK,2015
1363745487,GMT8141,NY,7/9/2015,21,P-U,LINCO,20160206,237,MADISON ST,BLK,2006
1363745517,GYK3760,NY,7/9/2015,21,SUBN,HONDA,20160709,201,HALSEY ST,OTHER,2006
1363745529,GYK3760,NY,7/9/2015,75,SUBN,HONDA,20160709,201,HALSEY ST,OTHER,2006
1363745542,GWL9925,NY,7/9/2015,71,SDN,TOYOT,20170602,482,TOMPKINS AVE,OTHER,2006
1363745554,GPH9963,PA,7/9/2015,21,SDN,MITSU,20150288,317,LEXINGTON AVE,OTHER,0


In [21]:
## Truncate the table
import pymysql # run pip install pymysql if this fails
conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True)
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("TRUNCATE TABLE `mdhaque_pvsubset`")
print "Table Truncate : " + str(time.time() - start)

Table Truncate : 1032.50184417


In [20]:
print ticketTable[0].keys()
print ticketTable[0]
dtl = row['Issue Date'].split("/")
print len(dtl)
dts = '0000-00-00 00:00'
dts = dtl[2] + '/' + dtl[0] + '/' + dtl[1] + ' ' + '00:00' 
print dts

['Street Name', 'House Number', 'Vehicle Body Type', 'Plate ID', 'Summons Number', 'Violation Code', 'Vehicle Year', 'Vehicle Color', 'Vehicle Make', 'Issue Date', 'Vehicle Expiration Date', 'Registration State']
{'Street Name': 'MACDOUNGH ST', 'House Number': '142', 'Vehicle Body Type': 'SDN', 'Plate ID': 'GGY6450', 'Summons Number': '1363745270', 'Violation Code': '46', 'Vehicle Year': '2010', 'Vehicle Color': 'WHITE', 'Vehicle Make': 'HONDA', 'Issue Date': '7/9/2015', 'Vehicle Expiration Date': '20170602', 'Registration State': '99'}
3
2015/7/20 00:00


In [ ]:
## Buffered insert to insert faster
i=0
sql=''
tokens = []
n=0

sql = '''INSERT INTO mdhaque_pvsubset (`summons_number`, `plate_id`, `registration_state`, `issue_date`, `violation_code`, 
  `vehicle_body_type`, `vehicle_make`, `vehicle_expiration_date`, `house_number`, `street_name`, `vehicle_color`, `vehicle_year`
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
blocksizes = [1000]
for bs in blocksizes:
    start = time.time()
    for row in ticketTable:
        dtl = row['Issue Date'].split("/")

        if len(dtl) == 3:
            dts = dts = dtl[2] + '/' + dtl[0] + '/' + dtl[1] + ' ' + '00:00' 
        tokens.append((row["Summons Number"],row["Plate ID"],row["Registration State"], dts, row["Violation Code"], \
        row["Vehicle Body Type"],row["Vehicle Make"], dts, row["House Number"], row["Street Name"], row["Vehicle Color"], row["Vehicle Year"]))
        #print conn.insert_id()
        if i % bs == 0:
            n+=1
            #print tokens
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            #print "Block " + str(n) +" "+ str(time.time() - bstart)
            tokens = []
        i+=1
    print "block size: " + str(bs) + " - total time : " + str(time.time() - start)
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()
conn.close()

In [ ]:
  `summons_number`, `plate_id`, `registration_state`, `issue_date`, `violation_code`, 
  `vehicle_body_type`, `vehicle_make`, `vehicle_expiration_date`, `house_number`,
  `street_name`, `vehicle_color`, `vehicle_year`

In [2]:
## At the begining if the file does not exist CREATE TABLE
import pymysql # run pip install pymysql if this fails

conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True) #setup credential from config file??
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute('''

CREATE TABLE `mdhaque_pvsubset` (
  `vehicle_id` int(5) NOT NULL AUTO_INCREMENT,
  `summons_number` varchar(10) NOT NULL,
  `plate_id` varchar(10) NOT NULL,
  `registration_state` varchar(2) NOT NULL,
  `issue_date` date NOT NULL,
  `violation_code` varchar(2) NOT NULL,
  `vehicle_body_type` varchar(10) NOT NULL,
  `vehicle_make` varchar(10) NOT NULL,
  `vehicle_expiration_date` date NOT NULL,
  `house_number` varchar(20) NOT NULL,
  `street_name` varchar(100) NOT NULL,
  `vehicle_color` varchar(20) NOT NULL,
  `vehicle_year` varchar(4) NOT NULL,
   PRIMARY KEY (`vehicle_id`)
) ENGINE=MyISAM DEFAULT CHARSET=latin1 ;
''')

cur.close()
conn.close()